In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
df = pd.read_pickle('/content/drive/My Drive/capstone/updated_processed.pkl')
df.head()

,Additional_Number_of_Scoring,Average_Score,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,reviews,Hotel_Country,review_month,Trip,Stayed nights,yr,test_reviews,test_positive_review,test_negative_review,reviews_processed,positive_reviews_processed,negative_reviews_processed,Combined_pos,Combined_neg,Combined_neu
0,241,8.2,138,2500,48,8,7.5,Bathtub and sink flush wasn t working properl...,4,8,1,6,2016,[bathtub and sink flush was not t working prop...,[we ve had a room with sea view and was lovely...,[bathtub and sink flush was not t working prop...,bathtub sink flush not working properly even t...,room sea view lovely location great want go be...,bathtub sink flush not working properly even t...,0.155,0.024,0.821
1,493,8.6,23,2865,9,3,5.4,The bedroom was tight and very small They exp...,3,12,0,2,2016,[the bedroom was tight and very small they exp...,[to be near to the main train station],[the bedroom was tight and very small they exp...,bedroom tight small explain hotel built old fa...,near main train station,bedroom tight small explain hotel built old fa...,0.000,0.000,1.000
2,908,8.5,0,3980,13,8,9.2,Great to have ipads in the room and the break...,5,2,1,1,2016,[great to have ipads in the room and the break...,[great to have ipads in the room and the break...,[no negative],great ipads room breakfast amazing,great ipads room breakfast amazing,negative,0.725,0.000,0.275
3,117,8.3,47,694,79,1,9.2,There was only one small issue with the room ...,1,7,1,6,2017,[there was only one small issue with the room ...,[the location was excellent it was off the cha...,[there was only one small issue with the room ...,one small issue room coffee machine come stand...,location excellent champ elysees minute walk a...,one small issue room coffee machine come stand...,0.405,0.000,0.595
4,300,9.0,68,1254,25,1,9.6,Price of breakfast was high We went to an ama...,5,3,1,1,2016,[price of breakfast was high we went to an ama...,[amazing location and quality of hotel being c...,[price of breakfast was high we went to an ama...,price breakfast high went amazing cafe road hi...,amazing location quality hotel close tube stat...,price breakfast high went amazing cafe road hi...,0.342,0.077,0.581


In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
nltk.download('vader_lexicon')
from nltk.sentiment.vader import SentimentIntensityAnalyzer
sid = SentimentIntensityAnalyzer()
from sklearn.feature_extraction.text import TfidfVectorizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...


/usr/local/lib/python3.6/dist-packages/nltk/twitter/__init__.py:20: UserWarning: The twython library has not been installed. Some functionality from the twitter package will not be available.
  warnings.warn("The twython library has not been installed. "


In [ ]:
stopwords_l = stopwords.words('english')
stopwords_l.remove('not')

In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import BaggingRegressor,RandomForestRegressor,AdaBoostRegressor
from sklearn import model_selection
from sklearn import metrics
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

In [ ]:
!pip install gensim

In [ ]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


True

In [ ]:
df.head(2)

,Additional_Number_of_Scoring,Average_Score,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,reviews,Hotel_Country,review_month,Trip,Stayed nights,yr,test_reviews,test_positive_review,test_negative_review,reviews_processed,positive_reviews_processed,negative_reviews_processed,Combined_pos,Combined_neg,Combined_neu
0,241,8.2,138,2500,48,8,7.5,Bathtub and sink flush wasn t working properl...,4,8,1,6,2016,[bathtub and sink flush was not t working prop...,[we ve had a room with sea view and was lovely...,[bathtub and sink flush was not t working prop...,bathtub sink flush not working properly even t...,room sea view lovely location great want go be...,bathtub sink flush not working properly even t...,0.155,0.024,0.821
1,493,8.6,23,2865,9,3,5.4,The bedroom was tight and very small They exp...,3,12,0,2,2016,[the bedroom was tight and very small they exp...,[to be near to the main train station],[the bedroom was tight and very small they exp...,bedroom tight small explain hotel built old fa...,near main train station,bedroom tight small explain hotel built old fa...,0.000,0.000,1.000


In [ ]:
df = df.drop(labels=['reviews','negative_reviews_processed','positive_reviews_processed','test_negative_review','test_reviews',
                     'test_positive_review'],axis=1)

In [ ]:
df.columns

Index(['Additional_Number_of_Scoring', 'Average_Score',
       'Review_Total_Negative_Word_Counts', 'Total_Number_of_Reviews',
       'Review_Total_Positive_Word_Counts',
       'Total_Number_of_Reviews_Reviewer_Has_Given', 'Reviewer_Score',
       'Hotel_Country', 'review_month', 'Trip', 'Stayed nights', 'yr',
       'reviews_processed', 'Combined_pos', 'Combined_neg', 'Combined_neu'],
      dtype='object')

In [ ]:
# step1: word tokenize of the 'reviews_processed'
from nltk.tokenize import word_tokenize
import nltk

corpus1 = []
    
for i in range(0, len(df['reviews_processed'])):
    words = [word.lower() for word in word_tokenize(df['reviews_processed'][i])]
    corpus1.append(words)
df['Tokenized_Combined'] = pd.Series(corpus1)

In [ ]:
df.head()

,Additional_Number_of_Scoring,Average_Score,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Hotel_Country,review_month,Trip,Stayed nights,yr,reviews_processed,Combined_pos,Combined_neg,Combined_neu,Tokenized_Combined
0,241,8.2,138,2500,48,8,7.5,4,8,1,6,2016,bathtub sink flush not working properly even t...,0.155,0.024,0.821,"[bathtub, sink, flush, not, working, properly,..."
1,493,8.6,23,2865,9,3,5.4,3,12,0,2,2016,bedroom tight small explain hotel built old fa...,0.000,0.000,1.000,"[bedroom, tight, small, explain, hotel, built,..."
2,908,8.5,0,3980,13,8,9.2,5,2,1,1,2016,great ipads room breakfast amazing,0.725,0.000,0.275,"[great, ipads, room, breakfast, amazing]"
3,117,8.3,47,694,79,1,9.2,1,7,1,6,2017,one small issue room coffee machine come stand...,0.405,0.000,0.595,"[one, small, issue, room, coffee, machine, com..."
4,300,9.0,68,1254,25,1,9.6,5,3,1,1,2016,price breakfast high went amazing cafe road hi...,0.342,0.077,0.581,"[price, breakfast, high, went, amazing, cafe, ..."


In [ ]:
#Applying Glove vectors on combined reviews: 
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [ ]:
# It is attaching the a unique sequence no. each tokenize word

tokenizer = Tokenizer(num_words=len(corpus1))
tokenizer.fit_on_texts(df['Tokenized_Combined'])

train_sequences = tokenizer.texts_to_sequences(df['Tokenized_Combined'])


In [ ]:
train_sequences[1]

[171, 983, 16, 1722, 4, 1904, 102, 274, 3368, 85, 205, 125, 31]

In [ ]:
# Its a dictionary which consists of unique word and its corresponding unique sequence no.
word_index = tokenizer.word_index
print("Number of unique words:", len(word_index))


Number of unique words: 20718


In [ ]:
word_index

4

In [ ]:
# embedding_dict = {}
# with open("/content/drive/MyDrive/capstone/glove.840B.300d.txt", "r",encoding="utf-8") as f:
#     for line in f:
#         values = line.split()
#         word = values[0]
#         # try:
#         #   vectors = np.asarray(values[1:], "float32")
#         #   embedding_dict[word] = vectors
#         # except ValueError:
#           # print(word)
#         print(values)
#         print(word)
#         print(len(values))
#         # break
# f.close()

In [ ]:
# # extracting the vector for each word i.e 300Dimensional from the file 
# embedding_dict = {}
# with open("/content/drive/MyDrive/capstone/glove.840B.300d.txt", "r",encoding="utf-8") as f:
#     for line in f:
#         values = line.split()
#         word = values[0]
#         try:
#           vectors = np.asarray(values[-300:], "float32")# -
#           embedding_dict[word] = vectors
#         except ValueError:
#           print(values)
#           print(word)
#           print(len(values))
#           print(values[-300:])
#           break
# f.close()

In [ ]:
# extracting the vector for each word i.e 200Dimensional from the file 
embedding_dict = {}
with open("/content/drive/MyDrive/capstone/glove.840B.300d.txt", "r",encoding="utf-8") as f:
    for line in f:
        values = line.split()
        word = values[0]
        vectors = np.asarray(values[-300:], "float32")
        embedding_dict[word] = vectors
f.close()

<!-- values = [bathtub,{300d}]
embedding_dict= {bathtub:300d,sink:300d} -->

In [ ]:
# 
num_words = len(word_index) + 1
embedding_matrix = np.zeros((num_words, 300))

for word, i in word_index.items():
    if i < num_words:
        emb_vec = embedding_dict.get(word)
        if emb_vec is not None:
            embedding_matrix[i] = emb_vec    # we are attaching the 300Dimensional vector of each word 

In [ ]:
word_index.items()

dict_items([('room', 1), ('staff', 2), ('not', 3), ('hotel', 4), ('location', 5), ('breakfast', 6), ('good', 7), ('great', 8), ('bed', 9), ('friendly', 10), ('nice', 11), ('helpful', 12), ('clean', 13), ('comfortable', 14), ('excellent', 15), ('small', 16), ('would', 17), ('could', 18), ('stay', 19), ('bathroom', 20), ('service', 21), ('nothing', 22), ('really', 23), ('bar', 24), ('one', 25), ('night', 26), ('u', 27), ('restaurant', 28), ('lovely', 29), ('everything', 30), ('station', 31), ('close', 32), ('time', 33), ('shower', 34), ('also', 35), ('view', 36), ('like', 37), ('well', 38), ('area', 39), ('bit', 40), ('reception', 41), ('day', 42), ('little', 43), ('get', 44), ('perfect', 45), ('food', 46), ('facility', 47), ('price', 48), ('check', 49), ('walk', 50), ('quiet', 51), ('free', 52), ('even', 53), ('amazing', 54), ('floor', 55), ('london', 56), ('comfy', 57), ('city', 58), ('coffee', 59), ('place', 60), ('need', 61), ('wifi', 62), ('minute', 63), ('much', 64), ('water', 65),

In [ ]:
len(embedding_dict),len(embedding_matrix)

(2195884, 20719)

In [ ]:
embedding_dict['hotel']

array([ 4.2714e-01,  1.2711e-01,  3.3711e-01, -1.0423e+00,  6.4144e-01,
       -1.2071e-01,  5.0131e-02, -4.7584e-01, -1.0312e-01,  2.9255e+00,
       -5.8213e-01, -3.2514e-01, -3.5034e-01,  2.1803e-01,  1.0048e-03,
       -8.2658e-01, -1.0869e-01,  1.4241e+00,  6.1642e-01, -4.4746e-02,
       -4.5506e-01,  4.3144e-01, -5.7901e-02, -1.7059e-02,  6.4482e-02,
       -6.0835e-02, -1.1422e+00, -9.9634e-02, -1.3181e-01,  6.7239e-01,
        9.1254e-01, -1.4970e-01,  3.3978e-01,  1.4313e-01, -2.8142e-01,
        4.5175e-02, -6.6405e-02, -5.2691e-01, -8.9802e-01, -2.1812e-01,
       -3.2022e-02, -4.4597e-01, -1.1959e-01, -8.5253e-02, -9.6036e-02,
        4.5640e-01,  4.3420e-01, -2.1666e-01, -3.3755e-01,  6.9133e-02,
       -5.0948e-01, -4.4991e-01,  4.8784e-01,  4.4107e-01, -1.9318e-01,
        4.7610e-02,  3.3381e-01, -1.3298e-01,  1.1601e-02,  2.4907e-01,
        7.3237e-01, -5.4820e-01,  1.7511e-01,  5.4786e-02, -4.9083e-01,
       -9.7755e-02, -2.0704e-01,  2.9365e-01,  2.0972e-02,  9.68

In [ ]:
embedding_matrix[4]
# 4.2714e-01,  1.2711e-01,  3.3711e-01, -1.0423e+00,  6.4144e-01,

array([ 4.27139997e-01,  1.27110004e-01,  3.37110013e-01, -1.04229999e+00,
        6.41439974e-01, -1.20710000e-01,  5.01310006e-02, -4.75840002e-01,
       -1.03119999e-01,  2.92549992e+00, -5.82130015e-01, -3.25139999e-01,
       -3.50340009e-01,  2.18030006e-01,  1.00479997e-03, -8.26579988e-01,
       -1.08690001e-01,  1.42410004e+00,  6.16419971e-01, -4.47460003e-02,
       -4.55060005e-01,  4.31439996e-01, -5.79009987e-02, -1.70590002e-02,
        6.44820035e-02, -6.08350001e-02, -1.14219999e+00, -9.96339992e-02,
       -1.31809995e-01,  6.72389984e-01,  9.12540019e-01, -1.49700001e-01,
        3.39780003e-01,  1.43130004e-01, -2.81419992e-01,  4.51750010e-02,
       -6.64049983e-02, -5.26910007e-01, -8.98020029e-01, -2.18119994e-01,
       -3.20219994e-02, -4.45969999e-01, -1.19589999e-01, -8.52530003e-02,
       -9.60360020e-02,  4.56400007e-01,  4.34199989e-01, -2.16659993e-01,
       -3.37550014e-01,  6.91329986e-02, -5.09480000e-01, -4.49910015e-01,
        4.87839997e-01,  

In [ ]:
# creating the 300Dimensional vector for each combined review by adding the 300Dimensional vector of each word in review
data_list = list()
for review in df['Tokenized_Combined']:
    sentence = np.zeros(300)
    count = 0
    for word in review:
        try:
            sentence += embedding_dict[word]   
            count += 1
        except KeyError:
            continue
    data_list.append(sentence/count)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: RuntimeWarning: invalid value encountered in true_divide
  if sys.path[0] == '':


In [ ]:
data_list[0]

array([ 1.29054360e-01,  7.97490073e-02, -5.04079696e-02, -1.10894074e-01,
        1.02033581e-01, -6.43482442e-02, -1.27210177e-02, -2.48734030e-01,
       -5.44678137e-02,  2.08470445e+00, -1.43763653e-01, -5.99888538e-02,
       -6.25697542e-03,  1.07806385e-02, -1.89667546e-01, -8.22586733e-02,
       -1.25822919e-01,  1.20787277e+00, -2.22414675e-03, -2.43816299e-02,
       -1.08932614e-01,  1.06985039e-01,  7.61997114e-03, -8.12601347e-02,
        9.53389894e-03, -3.23646429e-02, -1.27255666e-01, -8.85890993e-02,
       -1.00147012e-02,  1.51999871e-02,  3.56676520e-02, -2.68905111e-02,
       -9.65118915e-03,  9.86712366e-03,  2.05842640e-02, -5.72867970e-02,
        2.80610704e-02, -8.99613469e-02, -1.26674089e-01, -1.31121701e-01,
        4.67582322e-02,  3.14413697e-02, -5.23534830e-02,  5.79929894e-02,
       -5.75080628e-02,  1.45177278e-01, -4.72628814e-02, -1.02852348e-01,
       -4.13438234e-03,  9.97627644e-03, -7.80674360e-02, -4.57892115e-02,
        1.61673855e-01,  

In [ ]:
combined_review_df = pd.concat([df,pd.DataFrame(data_list)],axis=1)

In [ ]:
combined_review_df.shape,df.shape

((49677, 317), (49677, 17))

In [ ]:
# pkl the glove into the file
combined_review_df.to_pickle("/content/drive/My Drive/capstone/df_glove.pkl")

In [ ]:
df.head()

,Additional_Number_of_Scoring,Average_Score,Review_Total_Negative_Word_Counts,Total_Number_of_Reviews,Review_Total_Positive_Word_Counts,Total_Number_of_Reviews_Reviewer_Has_Given,Reviewer_Score,Hotel_Country,review_month,Trip,Stayed nights,yr,reviews_processed,Combined_pos,Combined_neg,Combined_neu,Tokenized_Combined
0,241,8.2,138,2500,48,8,7.5,4,8,1,6,2016,bathtub sink flush not working properly even t...,0.155,0.024,0.821,"[bathtub, sink, flush, not, working, properly,..."
1,493,8.6,23,2865,9,3,5.4,3,12,0,2,2016,bedroom tight small explain hotel built old fa...,0.000,0.000,1.000,"[bedroom, tight, small, explain, hotel, built,..."
2,908,8.5,0,3980,13,8,9.2,5,2,1,1,2016,great ipads room breakfast amazing,0.725,0.000,0.275,"[great, ipads, room, breakfast, amazing]"
3,117,8.3,47,694,79,1,9.2,1,7,1,6,2017,one small issue room coffee machine come stand...,0.405,0.000,0.595,"[one, small, issue, room, coffee, machine, com..."
4,300,9.0,68,1254,25,1,9.6,5,3,1,1,2016,price breakfast high went amazing cafe road hi...,0.342,0.077,0.581,"[price, breakfast, high, went, amazing, cafe, ..."
